# Scheldestromen

This script adds a new column "peilgebied_cat" and makes sure the peilgebieden allign with the HWS layer (Daniel):
- peilgebied_cat = 0 -> peilgebied
- peigelbied_cat = 1 -> RHWS (boezem)
- peilgebied_cat = 2 -> NHWS 

In [1]:
import geopandas as gpd
import numpy as np
from general_functions import *

%load_ext autoreload
%autoreload 2

remove_cat_2 = True

## Scheldestromen

In [2]:
#define relative paths
waterschap = 'Scheldestromen'
waterschap2 = 'Scheldestromen'

data_path = f"/DATAFOLDER/projects/4750_20/Data_postprocessed/Waterschappen/{waterschap}/{waterschap2}.gpkg"

# Waterschaps boundaries
grens_path = "/DATAFOLDER/projects/4750_30/Data_overig/Waterschapsgrenzen/Waterschapsgrenzen.geojson"
# Hoofdwatersysteem boundaries
hws_path = "/DATAFOLDER/projects/4750_30/Data_overig/HWS/krw_basins_vlakken.gpkg"
# Buffer boundaries
buffer_path =  r"/DATAFOLDER/projects/4750_30/Data_overig/HWS/hws_buffer_Scheldestromen.gpkg"
# Output folder
output_folder = f"/DATAFOLDER/projects/4750_30/Data_postprocessed/Waterschappen/{waterschap}"

### Load Files

In [3]:
# Load HHNK files
Scheldestromen = read_gpkg_layers(gpkg_path = data_path, 
                    variables = ['stuw', 
                                 'gemaal', 
                                 'hydroobject',
                                 'duikersifonhevel',
                                 'peilgebied', 
                                 'streefpeil',
                                 'aggregation_area',
                                ])
Scheldestromen['peilgebied'] = Scheldestromen['peilgebied'].to_crs('EPSG:28992')

# Load waterschap boundaries
gdf_grens = gpd.read_file(grens_path)
gdf_grens = gdf_grens.to_crs('EPSG:28992')
gdf_grens = gdf_grens.set_index('waterschap')

# Load hws
gdf_hws = gpd.read_file(hws_path)

# Load buffer
gdf_buffer = gpd.read_file(buffer_path)
gdf_buffer = gdf_buffer.to_crs('EPSG:28992')
gdf_buffer = gdf_buffer.dissolve()

In [4]:
Scheldestromen['peilgebied'].globalid.is_unique

True

In [5]:
Scheldestromen['peilgebied']

,code,nen3610id,globalid,geometry
0,GPG1398_dummy_id_0,dummy_nen3610id_peilgebied_0,dummy_globalid_peilgebied_0,MULTIPOLYGON Z (((39197.262 373523.013 -100000...
1,GPG1007_dummy_id_1,dummy_nen3610id_peilgebied_1,dummy_globalid_peilgebied_1,"MULTIPOLYGON Z (((15595.339 368349.545 0.000, ..."
2,GPG803_dummy_id_2,dummy_nen3610id_peilgebied_2,dummy_globalid_peilgebied_2,"MULTIPOLYGON Z (((36391.858 373284.887 0.000, ..."
3,GPG911_dummy_id_3,dummy_nen3610id_peilgebied_3,dummy_globalid_peilgebied_3,"MULTIPOLYGON Z (((40712.442 373620.190 0.000, ..."
4,GPG842_dummy_id_4,dummy_nen3610id_peilgebied_4,dummy_globalid_peilgebied_4,"MULTIPOLYGON Z (((33494.859 370960.048 0.000, ..."
...,...,...,...,...
847,GPG1333_dummy_id_847,dummy_nen3610id_peilgebied_847,dummy_globalid_peilgebied_847,"MULTIPOLYGON Z (((56639.793 391068.875 0.000, ..."
848,GPG1335_dummy_id_848,dummy_nen3610id_peilgebied_848,dummy_globalid_peilgebied_848,"MULTIPOLYGON Z (((57784.953 391743.719 0.000, ..."
849,GPG457_dummy_id_849,dummy_nen3610id_peilgebied_849,dummy_globalid_peilgebied_849,"MULTIPOLYGON Z (((22309.586 393311.125 0.000, ..."
850,GPG808_dummy_id_850,dummy_nen3610id_peilgebied_850,dummy_globalid_peilgebied_850,"MULTIPOLYGON Z (((58756.058 366653.895 0.000, ..."


## Select waterschap boundaries and clip hws layer

In [6]:
# Select boundaries HH Amstel, Gooi en Vecht
gdf_grens = gdf_grens.loc[['Waterschap Scheldestromen']]

# Use waterschap boudnaries to clip HWS layer
gdf_hws = gpd.overlay(gdf_grens, gdf_hws, how='intersection')

## Peilgebied and HWS layer overlap:
1. Identify the overlapping areas
2. Clip
3. Calculate overlapping area percentage
4. Filter

In [7]:
# Step 1: Identify the Overlapping Areas and clip
overlaps = gpd.overlay(Scheldestromen['peilgebied'], gdf_hws, how='intersection', keep_geom_type=True)

# # Step 2: Subtract Overlapping Areas from the original polygons in each DataFrame
non_overlapping_peilgebied = gpd.overlay(Scheldestromen['peilgebied'], overlaps, how='difference', keep_geom_type=True)
overlaps = gpd.overlay(non_overlapping_peilgebied, gdf_hws, how='intersection', keep_geom_type=False)
 
# Step 3: Calculate Area Percentages
# Calculate the area of overlaps
overlaps['overlap_area'] = overlaps.area

# Step 4: Filter based on area Area Percentages
minimum_area = 500
print(f'Number of overlapping shapes without filter: {len(overlaps)}')
overlap_ids = overlaps.loc[overlaps['overlap_area'] > minimum_area]
overlap_ids = overlap_ids.globalid.to_list()
print(f'Number of overlapping shapes with filter: {len(overlap_ids)}')

Number of overlapping shapes without filter: 203
Number of overlapping shapes with filter: 0


## Create peilgebied_cat columnm

In [8]:
# Add occurence to geodataframe
peilgebieden_cat = []

for index, row in Scheldestromen['peilgebied'].iterrows():
       
    if row.nen3610id == 'dummy_nen3610id_peilgebied_549':
        print(True)
        peilgebieden_cat.append(1)
    elif 'GPG437' in row.code:
        print('yes')
        peilgebieden_cat.append(1)
    elif 'dummy_code_nhws_3'in row.code:    
        peilgebieden_cat.append(1)
        print('yes2')
    else:
        peilgebieden_cat.append(0)

True
yes


In [9]:
# Add new column
Scheldestromen['peilgebied']['peilgebied_cat'] = peilgebieden_cat

In [10]:
Scheldestromen['peilgebied']['peilgebied_cat'].unique()

array([0, 1])

## Add nhws to ['peilgebied','streefpeil']

In [11]:
# update peilgebied dict key
gdf_hws['globalid'] = 'dummy_globalid_nhws_' + gdf_hws.index.astype(str)
gdf_hws['code'] = 'dummy_code_nhws_' + gdf_hws.index.astype(str)
gdf_hws['nen3610id'] = 'dummy_nen3610id_nhws_' + gdf_hws.index.astype(str)
gdf_hws['peilgebied_cat'] = 2

gdf_hws = gdf_hws[['globalid', 'code', 'nen3610id', 'peilgebied_cat', 'geometry']]

Scheldestromen['peilgebied'] = pd.concat([gdf_hws, Scheldestromen['peilgebied']])

In [12]:
# Create boezem streefpeil layer
streefpeil_hws = pd.DataFrame()
streefpeil_hws['waterhoogte'] = [np.nan] * len(gdf_hws)
streefpeil_hws['globalid'] = 'dummy_globalid_nhws_' + gdf_hws.index.astype(str)
streefpeil_hws['geometry'] = [None]* len(gdf_hws)

Scheldestromen['streefpeil'] = pd.concat([streefpeil_hws, Scheldestromen['streefpeil']])
Scheldestromen['streefpeil'] = gpd.GeoDataFrame(Scheldestromen['streefpeil'])

### Create buffer layer that ensures spatial match between peilgebied and hws layers based on the buffer layer

In [13]:
# # Create buffer polygon
# buffer_polygon = gpd.overlay(gdf_buffer, gdf_grens, how='intersection', keep_geom_type=True)
# buffer_polygon = gpd.overlay(buffer_polygon, gdf_hws, how='difference', keep_geom_type=True)
# buffer_polygon = gpd.overlay(buffer_polygon, Scheldestromen['peilgebied'], how='difference', keep_geom_type=True)

## Add buffer to ['peilgebied','streefpeil']

In [14]:
# # update peilgebied dict key
# buffer_polygon['globalid'] = 'dummy_globalid_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['code'] = 'dummy_code_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['nen3610id'] = 'dummy_nen3610id_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['peilgebied_cat'] = 2

# buffer_polygon = buffer_polygon[['globalid', 'code', 'nen3610id', 'peilgebied_cat', 'geometry']]

# Scheldestromen['peilgebied'] = pd.concat([buffer_polygon, Scheldestromen['peilgebied']])

In [15]:
# # Create boezem streefpeil layer
# streefpeil_buffer = pd.DataFrame()
# streefpeil_buffer['waterhoogte'] = [np.nan]
# streefpeil_buffer['globalid'] = ['dummy_globalid_nhws_buffer_1']
# streefpeil_buffer['geometry'] = [None]

# Scheldestromen['streefpeil'] = pd.concat([streefpeil_buffer, Scheldestromen['streefpeil']])
# Scheldestromen['streefpeil'] = gpd.GeoDataFrame(Scheldestromen['streefpeil'])

In [16]:
if remove_cat_2:
    Scheldestromen['peilgebied'] = Scheldestromen['peilgebied'].loc[Scheldestromen['peilgebied'].peilgebied_cat !=2]

## Store output

In [17]:
for key in Scheldestromen.keys():   
    print(key)
    Scheldestromen[str(key)].to_file(f'{output_folder}/{waterschap2}.gpkg', layer = str(key), driver='GPKG')

stuw
gemaal
hydroobject
duikersifonhevel
peilgebied
streefpeil
aggregation_area
